<h2>En este notebook, haremos una regresion lineal simple y comprobaremos como afectan al modelo los cambios de los hiperparametros (learning rate, epochs, batch size)</h2>

<br>

In [5]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [6]:
my_feature = [1.0, 2.0,  3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0, 10.0, 11.0, 12.0]
my_label = [5.0, 8.8,  9.6, 14.2, 18.8, 19.5, 21.4, 26.8, 28.9, 32.0, 33.8, 38.2]

In [7]:
px.scatter(x = my_feature,  y = my_label, labels = {'x' : 'Feature', 'y' : 'Label'})

In [8]:
# Creo mi modelo Secuencial
my_model =tf.keras.models.Sequential()

#units= Dimension de salida, input_shape dimension de entrada.
my_model.add(tf.keras.layers.Dense(units=1,input_shape=(1,)))
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


2022-08-23 13:01:42.696953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-23 13:01:42.696976: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-23 13:01:42.696995: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (esequiel-B550I-AORUS-PRO-AX): /proc/driver/nvidia/version does not exist
2022-08-23 13:01:42.697234: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
def create_model():
    my_model =tf.keras.models.Sequential()
    my_model.add(tf.keras.layers.Dense(units=1,input_shape=(1,)))
    return(my_model)

## Vemos que la cantidad de parametros entrenables son 2.
### Debido a que al ser una regresion lineal, lo que se tiene que construir es la recta:
<br>
<center>$\Huge\hat{y}$  $\LARGE =$  $\Huge w_{0}$ $\LARGE +$ $\Huge w_{1}$ $\LARGE *$ $\Huge x$</center>

##### Entonces los parametros a encontrar, son los que se denominan entrenables. Osea: $\Large w_{0}$ y $\Large w_{1}$

<br>

## Definamos ahora los hiperarametros:

In [10]:
learning_rate = 0.01 #que tan rapido se disminuye el error.
epochs = 10 #Cantidad de veces que se va a ver el bach de dataset
batch_size = 12

## Compilamos el modelo y lo entrenamos con los hiperparametros definidos previamente:

In [11]:
my_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate),
                 loss = 'mean_squared_error',
                 metrics = tf.keras.metrics.RootMeanSquaredError())

History = my_model.fit(x=my_feature,y=my_label,batch_size=batch_size,epochs=epochs)
trained_weight = my_model.get_weights()[0]
trained_bias = my_model.get_weights()[1]
hist = pd.DataFrame(History.history)
epochs_from_model = History.epoch
rmse = History.history["root_mean_squared_error"]

Epoch 1/10
1/1 [==============================] - 0s 144ms/step - loss: 383.6906 - root_mean_squared_error: 19.5880
Epoch 2/10
1/1 [==============================] - 0s 2ms/step - loss: 373.5421 - root_mean_squared_error: 19.3272
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 366.3113 - root_mean_squared_error: 19.1393
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 360.3344 - root_mean_squared_error: 18.9825
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 355.0852 - root_mean_squared_error: 18.8437
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 350.3203 - root_mean_squared_error: 18.7168
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: 345.9041 - root_mean_squared_error: 18.5985
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 341.7523 - root_mean_squared_error: 18.4865
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 337.8088 - root_

In [12]:
trained_weight[0][0]

0.73739225

In [13]:
trained_bias[0]

0.17112735

In [14]:
def fun(x,trained_weight,trained_bias):
    return(trained_bias + trained_weight * x)

def get_new_labels(trained_weight, trained_bias, my_features):
    return([fun(x,trained_weight[0][0], trained_bias[0]) for x in my_feature])

def plot_original_and_predicted_label(my_feature, my_label, model_label_output):
    fig = go.Figure()


    fig.add_trace(go.Scatter(x = my_feature, y = my_label,
                    mode ='markers', name = 'Original labels'))

    fig.add_trace(go.Scatter(x = my_feature, y = model_label_output,
                    mode = 'lines+markers',
                    name = 'Predicted labels'))

    fig.update_layout(title = 'Comparsion chart between predicted and original labels',
                 title_x = 0.5)
    fig.show()

def plot_loss(epochs_from_model, rmse):
    fig = px.line(x = epochs_from_model, y = rmse, labels = {'x' : 'Epochs', 'y' : 'Error'})
    fig.update_layout(title = 'Loss',
                 title_x = 0.5)
    fig.show()

In [15]:
model_label_output = get_new_labels(trained_weight, trained_bias, my_feature)

plot_original_and_predicted_label(my_feature, my_label, model_label_output)

In [16]:
plot_loss(epochs_from_model, rmse)

### Como se puede ver, la prediccion con los hiperparametros establecidos no es muy buena.  A simple vista se puede observar que tiene el error es alto.

## Cambiemos los hiperparametros y  volvemos a entrenar:

### Aumentemos los epochs:

In [17]:
learning_rate = 0.01 #que tan rapido se disminuye el error.
epochs = 300 #Cantidad de veces que se va a ver el bach de dataset
batch_size = 12

In [18]:
my_model = create_model()

my_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate),
                 loss = 'mean_squared_error',
                 metrics = tf.keras.metrics.RootMeanSquaredError())

History = my_model.fit(x=my_feature,y=my_label,batch_size=batch_size,epochs=epochs)
trained_weight = my_model.get_weights()[0]
trained_bias = my_model.get_weights()[1]
hist = pd.DataFrame(History.history)
epochs_from_model = History.epoch
rmse = History.history["root_mean_squared_error"]

Epoch 1/300
1/1 [==============================] - 0s 117ms/step - loss: 1278.4586 - root_mean_squared_error: 35.7555
Epoch 2/300
1/1 [==============================] - 0s 2ms/step - loss: 1259.8701 - root_mean_squared_error: 35.4946
Epoch 3/300
1/1 [==============================] - 0s 2ms/step - loss: 1246.5162 - root_mean_squared_error: 35.3060
Epoch 4/300
1/1 [==============================] - 0s 1ms/step - loss: 1235.4130 - root_mean_squared_error: 35.1484
Epoch 5/300
1/1 [==============================] - 0s 2ms/step - loss: 1225.6136 - root_mean_squared_error: 35.0088
Epoch 6/300
1/1 [==============================] - 0s 2ms/step - loss: 1216.6794 - root_mean_squared_error: 34.8809
Epoch 7/300
1/1 [==============================] - 0s 2ms/step - loss: 1208.3657 - root_mean_squared_error: 34.7616
Epoch 8/300
1/1 [==============================] - 0s 2ms/step - loss: 1200.5209 - root_mean_squared_error: 34.6485
Epoch 9/300
1/1 [==============================] - 0s 2ms/step - loss:

1/1 [==============================] - 0s 2ms/step - loss: 861.0432 - root_mean_squared_error: 29.3435
Epoch 72/300
1/1 [==============================] - 0s 1ms/step - loss: 856.3264 - root_mean_squared_error: 29.2631
Epoch 73/300
1/1 [==============================] - 0s 2ms/step - loss: 851.6230 - root_mean_squared_error: 29.1826
Epoch 74/300
1/1 [==============================] - 0s 1ms/step - loss: 846.9332 - root_mean_squared_error: 29.1021
Epoch 75/300
1/1 [==============================] - 0s 2ms/step - loss: 842.2568 - root_mean_squared_error: 29.0217
Epoch 76/300
1/1 [==============================] - 0s 2ms/step - loss: 837.5938 - root_mean_squared_error: 28.9412
Epoch 77/300
1/1 [==============================] - 0s 2ms/step - loss: 832.9440 - root_mean_squared_error: 28.8608
Epoch 78/300
1/1 [==============================] - 0s 1ms/step - loss: 828.3078 - root_mean_squared_error: 28.7803
Epoch 79/300
1/1 [==============================] - 0s 2ms/step - loss: 823.6849 - ro

1/1 [==============================] - 0s 1ms/step - loss: 563.0011 - root_mean_squared_error: 23.7276
Epoch 142/300
1/1 [==============================] - 0s 2ms/step - loss: 559.2134 - root_mean_squared_error: 23.6477
Epoch 143/300
1/1 [==============================] - 0s 1ms/step - loss: 555.4390 - root_mean_squared_error: 23.5678
Epoch 144/300
1/1 [==============================] - 0s 2ms/step - loss: 551.6779 - root_mean_squared_error: 23.4878
Epoch 145/300
1/1 [==============================] - 0s 2ms/step - loss: 547.9299 - root_mean_squared_error: 23.4079
Epoch 146/300
1/1 [==============================] - 0s 2ms/step - loss: 544.1951 - root_mean_squared_error: 23.3280
Epoch 147/300
1/1 [==============================] - 0s 2ms/step - loss: 540.4736 - root_mean_squared_error: 23.2481
Epoch 148/300
1/1 [==============================] - 0s 2ms/step - loss: 536.7651 - root_mean_squared_error: 23.1682
Epoch 149/300
1/1 [==============================] - 0s 1ms/step - loss: 533.0

1/1 [==============================] - 0s 1ms/step - loss: 329.6717 - root_mean_squared_error: 18.1569
Epoch 212/300
1/1 [==============================] - 0s 1ms/step - loss: 326.8044 - root_mean_squared_error: 18.0777
Epoch 213/300
1/1 [==============================] - 0s 2ms/step - loss: 323.9502 - root_mean_squared_error: 17.9986
Epoch 214/300
1/1 [==============================] - 0s 1ms/step - loss: 321.1090 - root_mean_squared_error: 17.9195
Epoch 215/300
1/1 [==============================] - 0s 1ms/step - loss: 318.2809 - root_mean_squared_error: 17.8404
Epoch 216/300
1/1 [==============================] - 0s 1ms/step - loss: 315.4659 - root_mean_squared_error: 17.7614
Epoch 217/300
1/1 [==============================] - 0s 2ms/step - loss: 312.6639 - root_mean_squared_error: 17.6823
Epoch 218/300
1/1 [==============================] - 0s 1ms/step - loss: 309.8750 - root_mean_squared_error: 17.6033
Epoch 219/300
1/1 [==============================] - 0s 2ms/step - loss: 307.0

1/1 [==============================] - 0s 1ms/step - loss: 160.3783 - root_mean_squared_error: 12.6641
Epoch 282/300
1/1 [==============================] - 0s 2ms/step - loss: 158.4189 - root_mean_squared_error: 12.5865
Epoch 283/300
1/1 [==============================] - 0s 2ms/step - loss: 156.4723 - root_mean_squared_error: 12.5089
Epoch 284/300
1/1 [==============================] - 0s 2ms/step - loss: 154.5385 - root_mean_squared_error: 12.4314
Epoch 285/300
1/1 [==============================] - 0s 1ms/step - loss: 152.6176 - root_mean_squared_error: 12.3538
Epoch 286/300
1/1 [==============================] - 0s 2ms/step - loss: 150.7094 - root_mean_squared_error: 12.2764
Epoch 287/300
1/1 [==============================] - 0s 1ms/step - loss: 148.8141 - root_mean_squared_error: 12.1989
Epoch 288/300
1/1 [==============================] - 0s 2ms/step - loss: 146.9316 - root_mean_squared_error: 12.1215
Epoch 289/300
1/1 [==============================] - 0s 1ms/step - loss: 145.0

In [19]:
model_label_output = get_new_labels(trained_weight, trained_bias, my_feature)

plot_original_and_predicted_label(my_feature, my_label, model_label_output)

In [20]:
plot_loss(epochs_from_model, rmse)

#### Vemos que el Loss disminuye mucho mas al aumentar los epochs.

### Veamos ahora aumentando el learning rate:

In [21]:
learning_rate = 0.3 #que tan rapido se disminuye el error.
epochs = 200 #Cantidad de veces que se va a ver el bach de dataset
batch_size = 12

In [22]:
my_model = create_model()
my_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate),
                 loss = 'mean_squared_error',
                 metrics = tf.keras.metrics.RootMeanSquaredError())

History = my_model.fit(x=my_feature,y=my_label,batch_size=batch_size,epochs=epochs)
trained_weight = my_model.get_weights()[0]
trained_bias = my_model.get_weights()[1]
hist = pd.DataFrame(History.history)
epochs_from_model = History.epoch
rmse = History.history["root_mean_squared_error"]

Epoch 1/200
1/1 [==============================] - 0s 110ms/step - loss: 1029.6537 - root_mean_squared_error: 32.0882
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 588.7039 - root_mean_squared_error: 24.2632
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 376.0023 - root_mean_squared_error: 19.3908
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 248.4287 - root_mean_squared_error: 15.7616
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 165.8610 - root_mean_squared_error: 12.8787
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 110.6748 - root_mean_squared_error: 10.5202
Epoch 7/200
1/1 [==============================] - 0s 1ms/step - loss: 73.3559 - root_mean_squared_error: 8.5648
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 48.1241 - root_mean_squared_error: 6.9372
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 31.2028 - 

1/1 [==============================] - 0s 1ms/step - loss: 5.3539 - root_mean_squared_error: 2.3138
Epoch 74/200
1/1 [==============================] - 0s 2ms/step - loss: 1.9981 - root_mean_squared_error: 1.4135
Epoch 75/200
1/1 [==============================] - 0s 1ms/step - loss: 1.2124 - root_mean_squared_error: 1.1011
Epoch 76/200
1/1 [==============================] - 0s 2ms/step - loss: 1.0056 - root_mean_squared_error: 1.0028
Epoch 77/200
1/1 [==============================] - 0s 2ms/step - loss: 0.9403 - root_mean_squared_error: 0.9697
Epoch 78/200
1/1 [==============================] - 0s 2ms/step - loss: 0.9164 - root_mean_squared_error: 0.9573
Epoch 79/200
1/1 [==============================] - 0s 1ms/step - loss: 0.9077 - root_mean_squared_error: 0.9527
Epoch 80/200
1/1 [==============================] - 0s 2ms/step - loss: 0.9066 - root_mean_squared_error: 0.9521
Epoch 81/200
1/1 [==============================] - 0s 1ms/step - loss: 0.9120 - root_mean_squared_error: 0.9

1/1 [==============================] - 0s 2ms/step - loss: 2.5339 - root_mean_squared_error: 1.5918
Epoch 146/200
1/1 [==============================] - 0s 2ms/step - loss: 2.4930 - root_mean_squared_error: 1.5789
Epoch 147/200
1/1 [==============================] - 0s 2ms/step - loss: 2.4399 - root_mean_squared_error: 1.5620
Epoch 148/200
1/1 [==============================] - 0s 2ms/step - loss: 2.3875 - root_mean_squared_error: 1.5451
Epoch 149/200
1/1 [==============================] - 0s 2ms/step - loss: 2.3459 - root_mean_squared_error: 1.5316
Epoch 150/200
1/1 [==============================] - 0s 2ms/step - loss: 2.3211 - root_mean_squared_error: 1.5235
Epoch 151/200
1/1 [==============================] - 0s 2ms/step - loss: 2.3156 - root_mean_squared_error: 1.5217
Epoch 152/200
1/1 [==============================] - 0s 1ms/step - loss: 2.3282 - root_mean_squared_error: 1.5258
Epoch 153/200
1/1 [==============================] - 0s 1ms/step - loss: 2.3552 - root_mean_squared_er

In [23]:
model_label_output = get_new_labels(trained_weight, trained_bias, my_feature)

plot_original_and_predicted_label(my_feature, my_label, model_label_output)

In [24]:
plot_loss(epochs_from_model, rmse)

#### Vemos que el loss, tiene altos y bajos antes de estabilizarse. Si lo trasladamos a la curva del gradiente descendiente, sabemos que estos altibajos, son en realidad que el lerning rate es lo suficientemente alto como para  que el valor pase de un lado de la curva al otro 

<br>

### Reduzcamos el batch_size:

In [25]:
learning_rate = 0.05 #que tan rapido se disminuye el error.
epochs = 300#Cantidad de veces que se va a ver el bach de dataset
batch_size = 6

In [26]:
my_model = create_model()
my_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate),
                 loss = 'mean_squared_error',
                 metrics = tf.keras.metrics.RootMeanSquaredError())

History = my_model.fit(x=my_feature,y=my_label,batch_size=batch_size,epochs=epochs)
trained_weight = my_model.get_weights()[0]
trained_bias = my_model.get_weights()[1]
hist = pd.DataFrame(History.history)
epochs_from_model = History.epoch
rmse = History.history["root_mean_squared_error"]

Epoch 1/300
2/2 [==============================] - 0s 1ms/step - loss: 719.1164 - root_mean_squared_error: 26.8163
Epoch 2/300
2/2 [==============================] - 0s 898us/step - loss: 621.2567 - root_mean_squared_error: 24.9250
Epoch 3/300
2/2 [==============================] - 0s 1ms/step - loss: 556.4400 - root_mean_squared_error: 23.5890
Epoch 4/300
2/2 [==============================] - 0s 872us/step - loss: 504.5598 - root_mean_squared_error: 22.4624
Epoch 5/300
2/2 [==============================] - 0s 946us/step - loss: 461.2827 - root_mean_squared_error: 21.4775
Epoch 6/300
2/2 [==============================] - 0s 1ms/step - loss: 423.4740 - root_mean_squared_error: 20.5785
Epoch 7/300
2/2 [==============================] - 0s 865us/step - loss: 389.1849 - root_mean_squared_error: 19.7278
Epoch 8/300
2/2 [==============================] - 0s 1ms/step - loss: 357.6471 - root_mean_squared_error: 18.9116
Epoch 9/300
2/2 [==============================] - 0s 946us/step - loss:

2/2 [==============================] - 0s 844us/step - loss: 1.0132 - root_mean_squared_error: 1.0066
Epoch 72/300
2/2 [==============================] - 0s 836us/step - loss: 1.1693 - root_mean_squared_error: 1.0814
Epoch 73/300
2/2 [==============================] - 0s 910us/step - loss: 0.9506 - root_mean_squared_error: 0.9750
Epoch 74/300
2/2 [==============================] - 0s 834us/step - loss: 0.9529 - root_mean_squared_error: 0.9761
Epoch 75/300
2/2 [==============================] - 0s 1ms/step - loss: 1.0140 - root_mean_squared_error: 1.0070
Epoch 76/300
2/2 [==============================] - 0s 702us/step - loss: 1.1932 - root_mean_squared_error: 1.0924
Epoch 77/300
2/2 [==============================] - 0s 814us/step - loss: 0.9294 - root_mean_squared_error: 0.9640
Epoch 78/300
2/2 [==============================] - 0s 890us/step - loss: 1.2230 - root_mean_squared_error: 1.1059
Epoch 79/300
2/2 [==============================] - 0s 751us/step - loss: 1.0549 - root_mean_sq

2/2 [==============================] - 0s 959us/step - loss: 0.9407 - root_mean_squared_error: 0.9699
Epoch 143/300
2/2 [==============================] - 0s 741us/step - loss: 0.8998 - root_mean_squared_error: 0.9486
Epoch 144/300
2/2 [==============================] - 0s 792us/step - loss: 1.0342 - root_mean_squared_error: 1.0169
Epoch 145/300
2/2 [==============================] - 0s 810us/step - loss: 1.1870 - root_mean_squared_error: 1.0895
Epoch 146/300
2/2 [==============================] - 0s 785us/step - loss: 0.9849 - root_mean_squared_error: 0.9924
Epoch 147/300
2/2 [==============================] - 0s 818us/step - loss: 0.8945 - root_mean_squared_error: 0.9458
Epoch 148/300
2/2 [==============================] - 0s 822us/step - loss: 0.9455 - root_mean_squared_error: 0.9724
Epoch 149/300
2/2 [==============================] - 0s 933us/step - loss: 1.0667 - root_mean_squared_error: 1.0328
Epoch 150/300
2/2 [==============================] - 0s 921us/step - loss: 0.9592 - ro

2/2 [==============================] - 0s 1ms/step - loss: 0.8841 - root_mean_squared_error: 0.9403
Epoch 213/300
2/2 [==============================] - 0s 891us/step - loss: 0.9110 - root_mean_squared_error: 0.9545
Epoch 214/300
2/2 [==============================] - 0s 857us/step - loss: 1.1450 - root_mean_squared_error: 1.0700
Epoch 215/300
2/2 [==============================] - 0s 803us/step - loss: 1.0499 - root_mean_squared_error: 1.0246
Epoch 216/300
2/2 [==============================] - 0s 834us/step - loss: 0.8927 - root_mean_squared_error: 0.9448
Epoch 217/300
2/2 [==============================] - 0s 899us/step - loss: 0.9590 - root_mean_squared_error: 0.9793
Epoch 218/300
2/2 [==============================] - 0s 1ms/step - loss: 0.9368 - root_mean_squared_error: 0.9679
Epoch 219/300
2/2 [==============================] - 0s 769us/step - loss: 1.0487 - root_mean_squared_error: 1.0241
Epoch 220/300
2/2 [==============================] - 0s 934us/step - loss: 0.9839 - root_m

2/2 [==============================] - 0s 818us/step - loss: 0.9617 - root_mean_squared_error: 0.9806
Epoch 283/300
2/2 [==============================] - 0s 936us/step - loss: 0.8860 - root_mean_squared_error: 0.9413
Epoch 284/300
2/2 [==============================] - 0s 798us/step - loss: 0.8766 - root_mean_squared_error: 0.9363
Epoch 285/300
2/2 [==============================] - 0s 972us/step - loss: 1.0094 - root_mean_squared_error: 1.0047
Epoch 286/300
2/2 [==============================] - 0s 768us/step - loss: 0.9100 - root_mean_squared_error: 0.9539
Epoch 287/300
2/2 [==============================] - 0s 865us/step - loss: 0.9011 - root_mean_squared_error: 0.9493
Epoch 288/300
2/2 [==============================] - 0s 979us/step - loss: 1.0610 - root_mean_squared_error: 1.0301
Epoch 289/300
2/2 [==============================] - 0s 937us/step - loss: 0.9388 - root_mean_squared_error: 0.9689
Epoch 290/300
2/2 [==============================] - 0s 869us/step - loss: 0.9032 - ro

In [27]:
model_label_output = get_new_labels(trained_weight, trained_bias, my_feature)

plot_original_and_predicted_label(my_feature, my_label, model_label_output)

In [28]:
plot_loss(epochs_from_model, rmse)

### Vemos que tenemos un caso similar al modelo anterior. Donde el gradiente descediente salta de un lado al otro de la curva.

<br>

### Encontremos un modelo que minimice el loss:

In [42]:
learning_rate = 0.05 #que tan rapido se disminuye el error.
epochs = 300#Cantidad de veces que se va a ver el bach de dataset
batch_size = 12

In [43]:
my_model = create_model()
my_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate),
                 loss = 'mean_squared_error',
                 metrics = tf.keras.metrics.RootMeanSquaredError())

History = my_model.fit(x=my_feature,y=my_label,batch_size=batch_size,epochs=epochs)
trained_weight = my_model.get_weights()[0]
trained_bias = my_model.get_weights()[1]
hist = pd.DataFrame(History.history)
epochs_from_model = History.epoch
rmse = History.history["root_mean_squared_error"]

Epoch 1/300
1/1 [==============================] - 0s 109ms/step - loss: 523.4875 - root_mean_squared_error: 22.8799
Epoch 2/300
1/1 [==============================] - 0s 2ms/step - loss: 465.5096 - root_mean_squared_error: 21.5757
Epoch 3/300
1/1 [==============================] - 0s 1ms/step - loss: 426.6869 - root_mean_squared_error: 20.6564
Epoch 4/300
1/1 [==============================] - 0s 2ms/step - loss: 396.0331 - root_mean_squared_error: 19.9006
Epoch 5/300
1/1 [==============================] - 0s 1ms/step - loss: 370.1476 - root_mean_squared_error: 19.2392
Epoch 6/300
1/1 [==============================] - 0s 2ms/step - loss: 347.4720 - root_mean_squared_error: 18.6406
Epoch 7/300
1/1 [==============================] - 0s 1ms/step - loss: 327.1453 - root_mean_squared_error: 18.0872
Epoch 8/300
1/1 [==============================] - 0s 2ms/step - loss: 308.6354 - root_mean_squared_error: 17.5680
Epoch 9/300
1/1 [==============================] - 0s 1ms/step - loss: 291.587

1/1 [==============================] - 0s 2ms/step - loss: 1.0460 - root_mean_squared_error: 1.0228
Epoch 73/300
1/1 [==============================] - 0s 2ms/step - loss: 1.0130 - root_mean_squared_error: 1.0065
Epoch 74/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9891 - root_mean_squared_error: 0.9945
Epoch 75/300
1/1 [==============================] - 0s 1ms/step - loss: 0.9721 - root_mean_squared_error: 0.9860
Epoch 76/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9603 - root_mean_squared_error: 0.9799
Epoch 77/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9521 - root_mean_squared_error: 0.9758
Epoch 78/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9465 - root_mean_squared_error: 0.9729
Epoch 79/300
1/1 [==============================] - 0s 1ms/step - loss: 0.9427 - root_mean_squared_error: 0.9709
Epoch 80/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9400 - root_mean_squared_error: 0.9

1/1 [==============================] - 0s 1ms/step - loss: 0.8769 - root_mean_squared_error: 0.9364
Epoch 145/300
1/1 [==============================] - 0s 2ms/step - loss: 0.8762 - root_mean_squared_error: 0.9361
Epoch 146/300
1/1 [==============================] - 0s 1ms/step - loss: 0.8760 - root_mean_squared_error: 0.9359
Epoch 147/300
1/1 [==============================] - 0s 2ms/step - loss: 0.8759 - root_mean_squared_error: 0.9359
Epoch 148/300
1/1 [==============================] - 0s 1ms/step - loss: 0.8759 - root_mean_squared_error: 0.9359
Epoch 149/300
1/1 [==============================] - 0s 2ms/step - loss: 0.8761 - root_mean_squared_error: 0.9360
Epoch 150/300
1/1 [==============================] - 0s 1ms/step - loss: 0.8767 - root_mean_squared_error: 0.9363
Epoch 151/300
1/1 [==============================] - 0s 2ms/step - loss: 0.8780 - root_mean_squared_error: 0.9370
Epoch 152/300
1/1 [==============================] - 0s 1ms/step - loss: 0.8816 - root_mean_squared_er

Epoch 216/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9166 - root_mean_squared_error: 0.9574
Epoch 217/300
1/1 [==============================] - 0s 1ms/step - loss: 0.9156 - root_mean_squared_error: 0.9568
Epoch 218/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9150 - root_mean_squared_error: 0.9566
Epoch 219/300
1/1 [==============================] - 0s 1ms/step - loss: 0.9151 - root_mean_squared_error: 0.9566
Epoch 220/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9156 - root_mean_squared_error: 0.9569
Epoch 221/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9165 - root_mean_squared_error: 0.9574
Epoch 222/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9176 - root_mean_squared_error: 0.9579
Epoch 223/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9186 - root_mean_squared_error: 0.9584
Epoch 224/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9193 - root_m

1/1 [==============================] - 0s 2ms/step - loss: 0.9176 - root_mean_squared_error: 0.9579
Epoch 288/300
1/1 [==============================] - 0s 1ms/step - loss: 0.9177 - root_mean_squared_error: 0.9579
Epoch 289/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9177 - root_mean_squared_error: 0.9580
Epoch 290/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9177 - root_mean_squared_error: 0.9580
Epoch 291/300
1/1 [==============================] - 0s 1ms/step - loss: 0.9177 - root_mean_squared_error: 0.9580
Epoch 292/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9178 - root_mean_squared_error: 0.9580
Epoch 293/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9178 - root_mean_squared_error: 0.9580
Epoch 294/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9178 - root_mean_squared_error: 0.9580
Epoch 295/300
1/1 [==============================] - 0s 2ms/step - loss: 0.9178 - root_mean_squared_er

In [44]:
model_label_output = get_new_labels(trained_weight, trained_bias, my_feature)

plot_original_and_predicted_label(my_feature, my_label, model_label_output)

In [45]:
plot_loss(epochs_from_model, rmse)

### Podemos ver que nuestro loss a partir de la epoch 250 empieza a converger.

<br>

## <ins>Algunas rules of thumb con respecto a los hiperparametros:</ins>

* ### Training loss should steadily decrease, steeply at first, and then more slowly until the slope of the curve reaches or approaches zero.

* ### If the training loss does not converge, train for more epochs.
    
* ### If the training loss decreases too slowly, increase the learning rate. Note that setting the learning rate too high may also prevent training loss from converging.
* ### If the training loss varies wildly (that is, the training loss jumps around), decrease the learning rate.
* ### Lowering the learning rate while increasing the number of epochs or the batch size is often a good combination.
* ### Setting the batch size to a very small batch number can also cause instability. First, try large batch size values. Then, decrease the batch size until you see degradation.
* ### For real-world datasets consisting of a very large number of examples, the entire dataset might not fit into memory. In such cases, you'll need to reduce the batch size to enable a batch to fit into memory.
